# Crowling Berita

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import uuid # Meskipun tidak digunakan untuk ID dari web, bisa untuk ID unik jika diperlukan

In [5]:
# URL homepage CNN Indonesia
url = "https://www.cnnindonesia.com/"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36 Edg/140.0.0.0"}

# Ambil halaman utama
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

data = []

# Loop semua link artikel di homepage
for artikel in soup.select("article a"):
    link = artikel.get("href")
    judul = artikel.get_text(strip=True)

    # Skip jika link kosong atau bukan http
    if not link or not link.startswith("http"):
        continue

    try:
        # Ambil halaman detail berita
        resp_detail = requests.get(link, headers=headers)
        soup_detail = BeautifulSoup(resp_detail.text, "html.parser")

        # --- PERUBAHAN DIMULAI DI SINI ---
        
        # Ambil ID Berita dari link
        id_berita = None
        try:
            # Contoh path: /ekonomi/20250910153012-92-1234567/nama-artikel
            # Kita ambil bagian sebelum nama artikel, yaitu '20250910153012-92-1234567'
            id_segment = link.split("/")[-2]
            # Kemudian kita pisah dengan '-' dan ambil bagian terakhirnya
            id_berita = id_segment.split("-")[-1]
        except (IndexError, AttributeError):
            # Jika struktur URL berbeda dan gagal, ID akan tetap None
            id_berita = None

        # Ambil kategori dari link (path pertama setelah domain)
        kategori = None
        try:
            path = link.replace("https://www.cnnindonesia.com/", "")
            kategori = path.split("/")[0] if path else None
        except Exception:
            kategori = None

        # Ambil isi berita
        paragraf = [p.get_text(strip=True) for p in soup_detail.select("div.detail-text p")]
        isi = " ".join(paragraf)

        if isi:  # hanya simpan kalau ada isi berita
            data.append({
                "id_berita": id_berita, # <-- Kolom baru ditambahkan di sini
                "judul": judul,
                "kategori": kategori,
                "isi": isi,
                "link": link
            })
            
        # --- PERUBAHAN SELESAI DI SINI ---

    except Exception as e:
        print(f"Gagal ambil {link}: {e}")

# Simpan ke dataframe Pandas
df = pd.DataFrame(data)

# Simpan ke file CSV
df.to_csv("berita_cnn_dengan_id.csv", index=False, encoding="utf-8-sig")

# Tampilkan hasil di notebook
pd.set_option("display.max_colwidth", 100)  # biar isi tidak kepotong
print("Proses selesai. Berikut adalah contoh datanya:")
display(df)

Proses selesai. Berikut adalah contoh datanya:


,id_berita,judul,kategori,isi,link
0,1271886,Komisi III DPR Mulai Rapat Maraton Seleksi Calon Hakim Agung,nasional,Komisi III DPR RI mulai menggelar rapat maraton untuk melakukan seleksi atau uji kelayakan dan k...,https://www.cnnindonesia.com/nasional/20250910041637-32-1271886/komisi-iii-dpr-mulai-rapat-marat...
1,1271880,Rieke PDIP Usul Prabowo Terbitkan Perpres Jaminan Sosial buat Ojol,nasional,"Anggota Komisi VI DPR dari FraksiPDIP, Rieke Diah Pitaloka alias Oneng meminta Presiden Prabowo ...",https://www.cnnindonesia.com/nasional/20250910030753-32-1271880/rieke-pdip-usul-prabowo-terbitka...
2,1272266,01Sara Keponakan Prabowo Mundur dari DPRNasional,nasional,Politikus Partai GerindraRahayu Saraswati Djojohadikusumoalias Sara memutuskan mundur sebagai an...,https://www.cnnindonesia.com/nasional/20250910182921-32-1272266/sara-keponakan-prabowo-mundur-da...
3,1272212,02Purbaya Sebut Rp425 T Mengendap di BI: Makanya Orang Susah Cari KerjaEkonomi,ekonomi,Menteri KeuanganPurbaya Yudhi Sadewamenyebut duit pemerintah sekitar Rp425 triliun mengendap diB...,https://www.cnnindonesia.com/ekonomi/20250910160549-532-1272212/purbaya-sebut-rp425-t-mengendap-...
4,1272167,"03Israel Serang Doha Qatar Kerahkan 15 Jet Tempur, Tembakkan 10 RudalInternasional",internasional,Israeldiklaim mengerahkan 15 jet tempur dan menembakkan sedikitnya 10 rudal untuk menyerang area...,https://www.cnnindonesia.com/internasional/20250910150426-120-1272167/israel-serang-doha-qatar-k...
5,1272248,"04Update Korban Banjir di Bali: 2 Dilaporkan Tewas, 8 Masih HilangNasional",nasional,"Kepala Kantor Pencarian dan Pertolongan Denpasar, I Nyoman Sidakarya mengatakan 9 orang yang ter...",https://www.cnnindonesia.com/nasional/20250910173714-20-1272248/update-korban-banjir-di-bali-2-d...
6,1272165,"05Ada Putusan MK, TNI Masih Pertimbangkan Polisikan Ferry IrwandiNasional",nasional,TNImasih mempertimbangkan langkah hukum yang akan diambil terhadap CEO Malaka ProjectFerry Irwan...,https://www.cnnindonesia.com/nasional/20250910145755-12-1272165/ada-putusan-mk-tni-masih-pertimb...
7,1272135,"06Berstatus Terpidana, Silfester Matutina Masih Jadi Komisaris ID FoodEkonomi",ekonomi,"Silfester Matutina, terpidana kasus dugaan pencemaran nama baik terhadap Jusuf Kalla, ternyata m...",https://www.cnnindonesia.com/ekonomi/20250910135552-92-1272135/berstatus-terpidana-silfester-mat...
8,1271984,"Bali Diterjang Banjir Besar, Dua Warga Meninggal Dunia",nasional,"Banjirbesar melandaBali, menerjang sejumlah daerah seperti Kabupaten Jembrana dan Kota Denpasar ...",https://www.cnnindonesia.com/nasional/20250910093338-20-1271984/bali-diterjang-banjir-besar-dua-...
9,1271909,"Timnas Indonesia era Kluivert: Inverted Fullback, Ball Winner, Mezzala",olahraga,Tak berlebihan kiranya jikaPatrick Kluivertdisebut mulai memperlihatkan jati dirinya diTimnas In...,https://www.cnnindonesia.com/olahraga/20250910061945-142-1271909/timnas-indonesia-era-kluivert-i...
